In [9]:
from __future__ import annotations
"""
Collect aggregate stats CKA etc... from everything.

TODO(Adriano) 1st do without the controls, THEN do with the controls.
"""
from pathlib import Path

embeddings_paths = [Path("/mnt/align3_drive/adrianoh/dl_final_project_embeddings")]
paths_ols_linear = [Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets")]
paths_gd_mlp = [
    Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/arguana_hf_cartesian_product"),
    Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/nfcorpus_hf_cartesian_product"),
    Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/scidocs_hf_cartesian_product"),
    Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/trec-covid_hf_cartesian_product"),
    Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/fiqa_hf_cartesian_product"),
    Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/hotpotqa_hf_cartesian_product"),
    Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/dl_final_project_layers_2plus_v1")
]

In [19]:
"""
Shit here is copied from `pokemon.py` and `quadrocopter_evolved.ipynb`.
"""

from typing import List
################ GET MODEL FILES ################
# TODO: not clear what is going on here exactly
# def __get_model_files(model_dir: Path) -> List[Path]:
#     return  [
#             # embeddings
#             model_dir / f"embeddings_corpus_train.safetensors",
#             model_dir / f"embeddings_corpus_validation.safetensors",
#             model_dir / f"embeddings_queries_train.safetensors",
#             model_dir / f"embeddings_queries_validation.safetensors",
#             # metadata
#             model_dir / f"metadatas_corpus_train.jsonl",
#             model_dir / f"metadatas_corpus_validation.jsonl",
#             model_dir / f"metadatas_queries_train.jsonl",
#             model_dir / f"metadatas_queries_validation.jsonl",
#         ]
# def __get_reversed_model_files(model_dir: Path) -> List[Path]:
#     return [
#         # reverse because sometimes we do "corpus_embeddings..."
#         #
#         # saftensors
#         model_dir / f"corpus_train_embeddings.safetensors",
#         model_dir / f"queries_train_embeddings.safetensors",
#         model_dir / f"corpus_validation_embeddings.safetensors",
#         model_dir / f"queries_validation_embeddings.safetensors",
#         # jsonls
#         model_dir / f"corpus_train_metadatas.jsonl",
#         model_dir / f"queries_train_metadatas.jsonl",
#         model_dir / f"corpus_validation_metadatas.jsonl",
#         model_dir / f"queries_validation_metadatas.jsonl",
#     ]

# def get_model_files(model_dir: Path) -> List[Path]:
#     files = __get_model_files(model_dir)
#     if not all(f.exists() for f in files):
#         files = __get_reversed_model_files(model_dir)
#     assert all(f.exists() for f in files), f"Files {files} do not exist in {model_dir}"
#     return files

################ GET EMBEDDINGS PATHS ################
def get_embeddings_paths(embeddings_path: Path):
    record_type = "corpus"
    embeddings_train_path = embeddings_path / f"embeddings_{record_type}_train.safetensors"
    embeddings_validation_path = embeddings_path / f"embeddings_{record_type}_validation.safetensors"
    assert (
        (embeddings_train_path.exists() and embeddings_validation_path.exists()) or 
        (not embeddings_train_path.exists() and not embeddings_validation_path.exists())
    )
    if not embeddings_train_path.exists():
        # NOTE: that sometimes the path names are reversed, i.e. when using OpenAI models; you can observe
        # more in detail in `get_reversed_model_files` in `sanity_check_embeddings_note_equal.ipynb`
        embeddings_train_path = embeddings_path / f"{record_type}_train_embeddings.safetensors"
        embeddings_validation_path = embeddings_path / f"{record_type}_validation_embeddings.safetensors"
    assert embeddings_train_path.exists() and embeddings_validation_path.exists(), f"Files {embeddings_train_path} and {embeddings_validation_path} do not exist" # fmt: skip
    return embeddings_train_path, embeddings_validation_path

def get_all_embeddings(embeddings_path: Path, dataset: str, device: str) -> Dict[str, torch.Tensor]:
    model2embeddings: Dict[str, torch.Tensor] = {}
    for model_name in MODEL_NAMES:
        embeddings_path_src_parent  = embeddings_path / model_name.replace("/", "_") / dataset
        embeddings_path_src_train, _ = get_embeddings_paths(embeddings_path_src_parent)
        embeddings_src_train = load_file(embeddings_path_src_train)["embeddings"].to(device)
        model2embeddings[model_name] = embeddings_src_train
    return model2embeddings

################ models and constants ################
from __future__ import annotations
import click
import random
import math
import gc
import torch
import einops
import numpy as np
import time
import wandb
import torch.nn as nn
from typing import List, Optional, Dict, Literal, Tuple
from pathlib import Path
from pydantic import BaseModel
from safetensors.torch import load_file, save_file
from torch.utils.data import Dataset, DataLoader
import tqdm

DATASETS = [
    # (numbers are counts for documents, there may be some longer documents -> slightly more chunks)
    "arguana", # 10K
    "fiqa", # 50K -> 20K
    "scidocs", # 25K -> 20K
    "nfcorpus", # 5K
    "hotpotqa", # 100K -> 20K
    "trec-covid", # too much -> 20K
]

MODEL_NAMES = [
    "WhereIsAI/UAE-Large-V1",
    "BAAI/bge-base-en-v1.5",
    "BAAI/bge-large-en-v1.5",
    "BAAI/bge-small-en-v1.5",
    "intfloat/e5-base-v2",
    "intfloat/e5-large-v2",
    "intfloat/e5-small-v2",
    "thenlper/gte-base",
    "thenlper/gte-large",
    "thenlper/gte-small",
    "sentence-transformers/gtr-t5-base",
    "sentence-transformers/gtr-t5-large",
    "mixedbread-ai/mxbai-embed-large-v1",
    "sentence-transformers/sentence-t5-base",
    "sentence-transformers/sentence-t5-large",
    "openai/text-embedding-3-large",
    "openai/text-embedding-3-small",
]
N_LAYERS = list(range(0, 8)) # 0 -> OLS

class StitchPair(BaseModel):
    source: str
    target: str
    dataset: str
    num_layers: Optional[int] = None
    # NOTE: first layer size SHOULD be input dim and last layer SHOULD size be output dim
    layer_dims: Optional[List[int]] = None
    mode: Literal["affine", "mlp", "affine_ols"] = "mlp"

class MLP(nn.Module):
    def __init__(
        self, 
        input_dim: int,
        output_dim: int,
        source_model_name: str,
        target_model_name: str,
        dataset: str,
        hidden_dims: Optional[List[int]] = None,
        num_layers: int = 1
    ):
        super().__init__()
        num_layers = num_layers or len(hidden_dims)
        if num_layers is None:
            raise ValueError("Either num_layers or hidden_dims must be provided")
        if hidden_dims is None:
            hidden_dims = [max(input_dim, output_dim)] * (num_layers - 1)
            
        # Build layer dimensions including input and output
        layer_dims = [input_dim] + hidden_dims + [output_dim]
        
        # Create sequential model with linear layers and ReLU activations
        layers: List[nn.Module] = []
        for i in range(len(layer_dims)-1):
            layers.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
            if i < len(layer_dims)-2:  # No ReLU after final layer
                layers.append(nn.ReLU())
                
        self.model = nn.Sequential(*layers)
        self.source_model_name = source_model_name
        self.target_model_name = target_model_name
        self.dataset = dataset
        self.num_layers = num_layers
        self.layer_dims = layer_dims
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)
    
    def save_to_folder_path(self, save_path: Path):
        assert not save_path.is_file(), f"Save path {save_path} is a file, not a directory"
        save_path.mkdir(parents=True, exist_ok=True)
        model_file = save_path / f"mlp.safetensors"
        info_file = save_path / f"stitch_info.json"
        assert not model_file.exists(), f"Model file already exists at {model_file}"
        assert not info_file.exists(), f"Info file already exists at {info_file}"
        save_file(self.state_dict(), model_file)
        info_file.write_text(
            StitchPair(
                source=self.source_model_name,
                target=self.target_model_name,
                dataset=self.dataset,
                mode="mlp",
                num_layers=self.num_layers,
                layer_dims=self.layer_dims
            ).model_dump_json()
        )
    
    @staticmethod
    def load_from_folder_path(load_path: Path):
        assert load_path.is_dir(), f"Load path {load_path} is not a directory"
        model_file = load_path / f"mlp.safetensors"
        info_file = load_path / f"stitch_info.json"
        assert model_file.exists(), f"Model file does not exist at {model_file}"
        assert info_file.exists(), f"Info file does not exist at {info_file}"
        pd_model = StitchPair.model_validate_json(info_file.read_text())
        hidden_dims = pd_model.layer_dims[1:-1]
        return MLP(
            input_dim=pd_model.layer_dims[0],
            output_dim=pd_model.layer_dims[-1],
            source_model_name=pd_model.source,
            target_model_name=pd_model.target,
            dataset=pd_model.dataset,
            hidden_dims=hidden_dims,
            num_layers=pd_model.num_layers,
        )
    
    def __repr__(self):
        return f"MLP(input_dim={self.input_dim}, output_dim={self.output_dim}, hidden_dims={self.hidden_dims}, num_layers={self.num_layers})"


def model2model_dimension(model_name: str) -> int:
    """ Helper: get the size of the embedding dimension vector (1D, usually something like 768-4096). """
    # Miscellaneous (HF)
    if "/" in model_name:
        assert model_name.count("/") == 1
        model_name = model_name.split("/")[-1]
    if model_name == "SFR-Embedding-Mistral":
        return 4096
    elif model_name == "UAE-Large-V1":
        return 1024
    elif model_name == "mxbai-embed-large-v1":
        return 1024
    # BGE Models (HF)
    elif model_name == "bge-base-en-v1.5":
        return 768
    elif model_name == "bge-large-en-v1.5":
        return 1024
    elif model_name == "bge-small-en-v1.5":
        return 384
    #  E5 Models (HF)
    elif model_name == "e5-base-v2":
        return 768
    elif model_name == "e5-large-v2":
        return 1024
    elif model_name == "e5-small-v2":
        return 384
    # GTE Models (HF)
    elif model_name == "gte-base":
        return 768
    elif model_name == "gte-large":
        return 1024
    elif model_name == "gte-small":
        return 384
    # GTR-T5 Models (HF)
    elif model_name == "gtr-t5-base":
        return 768
    elif model_name == "gtr-t5-large":
        return 768
    # Sentence T5 (HF)
    elif model_name == "sentence-t5-base":
        return 768
    elif model_name == "sentence-t5-large":
        return 768
    # OpenAI Models
    elif model_name == "text-embedding-3-large":
        return 3072
    elif model_name == "text-embedding-3-small":
        return 1536
    else:
        # NOTE: cohere may be supported in THE FUTURE
        raise ValueError(f"Unsupported model: {model_name}")

In [16]:
"""
First, be able to get all the models.
"""

def get_all_model(stitch_info_parent_path: Path) -> MLP:
    stitch_info_file = stitch_info_parent_path / "stitch_info.json"
    assert stitch_info_file.exists(), f"Stitch info file does not exist at {stitch_info_file}"
    stitch_info = StitchPair.model_validate_json(stitch_info_file.read_text())
    if stitch_info.mode == "mlp":
        return MLP.load_from_folder_path(stitch_info_parent_path)
    else:
        assert stitch_info.mode in ["affine_ols", "affine", "mlp"], f"Unsupported mode: {stitch_info.mode}"
        src_dim: int = model2model_dimension(stitch_info.source)
        dst_dim: int = model2model_dimension(stitch_info.target)
        mlp = MLP(
            input_dim=src_dim,
            output_dim=dst_dim,
            source_model_name=stitch_info.source,
            target_model_name=stitch_info.target,
            dataset=stitch_info.dataset,
            hidden_dims=[],
            num_layers=1
        )
        state_dict_file = (
            stitch_info_parent_path / "linear_transform.safetensors" if stitch_info.mode == "affine_ols" else 
            stitch_info_parent_path / "checkpoints" / "checkpoint_40.safetensors"
        )
        assert state_dict_file.exists(), f"State dict file does not exist at {state_dict_file}"
        state_dict = load_file(state_dict_file)
        assert len(mlp.model) == 1
        weights = state_dict["weights"] if "weights" in state_dict else state_dict["weight"]
        biases = state_dict["biases"] if "biases" in state_dict else state_dict["bias"]
        mlp.model[0].weight.data = weights
        mlp.model[0].bias.data = biases
        return mlp

ols_paths: List[Path] = []
for path in tqdm.tqdm(paths_ols_linear):
    relevant_paths_ = list(set(x.parent for x in list(path.glob("**/stitch_info.json")) + list(path.glob("**/stitch_info_pairs.json"))))
    ols_paths.extend(relevant_paths_)
print('\n'.join(x.as_posix() for x in ols_paths))
mlps = [get_all_model(path) for path in tqdm.tqdm(ols_paths)]
# mlp_paths: List[Path] = []
# for path in paths_gd_linear:
#     mlp_paths.append(path / "embeddings")
# for path in paths_non_linear:
#     mlp_paths.append(path / "embeddings")







100%|██████████| 1/1 [00:02<00:00,  2.21s/it]


/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/thenlper_gte-small_intfloat_e5-large-v2/scidocs
/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/BAAI_bge-large-en-v1.5_thenlper_gte-base/nfcorpus
/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/sentence-transformers_gtr-t5-large_intfloat_e5-base-v2/arguana
/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/sentence-transformers_sentence-t5-base_mixedbread-ai_mxbai-embed-large-v1/trec-covid
/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/mixedbread-ai_mxbai-embed-large-v1_intfloat_e5-base-v2/fiqa
/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/thenlper_gte-small_sentence-transformers_gtr-t5-base/trec-covid
/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/sentence-transformers_gtr-t5-base_intfloat_e5-large-v2/scidocs
/mnt/align3_drive/adrianoh/dl_final_project_layers/ols_all_datasets/openai_text-embedding-3









































































































































































  0%|          | 0/1632 [01:40<?, ?it/s]












































































































































































































100%|██████████| 1632/1632 [00:13<00:00, 118.73it/s]


In [63]:
import click
import numpy as np
import plotly.express as px
import random
import torch

from sklearn.neighbors import NearestNeighbors
from torch.nn.functional import cosine_similarity, pairwise_distance

################################ COPIED FROM THE ORIGINAL PAPER ################################
COSINE = "cosine"
EUCLIDEAN = "euclidean"
JACCARD = "jaccard"
RANK = "rank"
CKA = "cka"
AVAILABLE_METRICS = [COSINE, EUCLIDEAN, JACCARD, RANK, CKA]
MATCH_DIM_METRICS = [COSINE, EUCLIDEAN]
NEAREST_NEIGHBORS = [JACCARD, RANK]

def _pairwise_similarity(
    embeds1: torch.Tensor, embeds2: torch.Tensor, metric: str, device: torch.device
):
    """
    Calculates the pairwise cosine similarity or Euclidean distance between the given tensors.

    :param embeds1: The first tensors of dimension (N, D).
    :param embeds2: The second tensors of dimension (N, D).
    :param metric: The metric to be used.
    :param device: The device on which calculations are performed.
    :return: The calculated pairwise scores.
    """
    if metric == COSINE:
        return cosine_similarity(embeds1.to(device), embeds2.to(device))
    elif metric == EUCLIDEAN:
        return pairwise_distance(embeds1.to(device), embeds2.to(device))
    else:
        raise NotImplementedError(f"Provided unsupported metric {metric} for pairwise similarity!")


def _mean_pairwise_similarity(
    embeds1: torch.Tensor, embeds2: torch.Tensor, metric: str, batch_size: int, device: torch.device
):
    """
    Calculates the pairwise cosine similarity or Euclidean distance between batches of the given tensors and sums them
    up. In the end, the mean score is returned.

    :param embeds1: A tensor of dimension (N, D).
    :param embeds2: A tensor of dimension (N, D).
    :param metric: The metric to use for calculating the pairwise scores.
    :param batch_size: The batch size used to split the tensors.
    :param device: The device on which calculations should be performed.
    :return: The mean score over all batches.
    """
    assert embeds1.shape == embeds2.shape
    embeds1_batches = embeds1.split(batch_size)
    embeds2_batches = embeds2.split(batch_size)
    sum = 0
    sims = []
    for embed1, embed2 in zip(embeds1_batches, embeds2_batches):
        sim = _pairwise_similarity(embed1, embed2, metric, device)
        sims = sims + sim.detach().cpu().tolist()
        sum += torch.sum(sim).detach().cpu()
    text = "Cosine Similarities" if metric is COSINE else "Euclidean Distance"
    fig = px.histogram(x=sims, labels={"x": text}, title=f"{text} Distribution")
    return sum / len(embeds1), fig


def _jaccard_sim(indices1: np.ndarray, indices2: np.ndarray):
    """
    Calculates the Jaccard similarity between two 2D arrays by dividing the number of overlapping entries by the union
    of entries per row. The mean score over all rows is returned in the end.

    :param indices1: The first array of indices of shape (N, D).
    :param indices2: The second array of indices of shape (N, D).
    :return: The mean Jaccard similarity.
    """
    inds = np.concatenate((indices1, indices2), axis=1)
    len_union = np.array([len(np.unique(i)) for i in inds])
    len_intersection = np.array(
        [len(set(i).intersection(set(j))) for i, j in zip(indices1, indices2)]
    )
    return np.mean(len_intersection / len_union)


def _get_rank_sum(indices1: np.ndarray, indices2: np.ndarray):
    """
    Computes the sum term for rank similarity given the two 1D-arrays containing the indices of the k-nearest neighbors
    of two sets of activations.

    :param indices1: One row of indices calculated for the first set of activations.
    :param indices2: One row of indices calculated for the second set of activations.
    :return: The calculated rank sum.
    """
    aux = np.concatenate((indices1, indices2))
    aux_sort_indices = np.argsort(aux, kind="mergesort")
    aux = aux[aux_sort_indices]
    mask = aux[1:] == aux[:-1]
    ar1_indices = aux_sort_indices[:-1][mask] + 1
    ar2_indices = aux_sort_indices[1:][mask] - indices1.size + 1
    rank_sum = np.sum([2 / ((1 + abs(i - j)) * (i + j)) for i, j in zip(ar1_indices, ar2_indices)])
    return rank_sum


def _rank_sim(indices1: np.ndarray, indices2: np.ndarray):
    """
    Computes the rank similarity between two sets of indices. Rank similarities are calculated for each pair of rows and
    averaged.

    :param indices1: The first array of indices of shape (N, D).
    :param indices2: The second array of indices of shape (N, D).
    :return: The mean rank similarity.
    """
    rank_sums = [_get_rank_sum(i, j) for i, j in zip(indices1, indices2)]
    len_intersection = np.array(
        [len(set(i).intersection(set(j))) for i, j in zip(indices1, indices2)]
    )
    factors = []
    for idx, elem1 in enumerate(len_intersection):
        if elem1 > 0:
            factors.append(1 / sum([1 / (i + 1) for i in range(int(elem1))]))
        else:
            factors.append(0)
    res = np.array(factors) * np.array(rank_sums)
    return np.mean(res)


def nn_sim(indices1: np.ndarray, indices2: np.ndarray, metric: str):
    """
    Calculates the similarity of two sets of indices representing the index of nearest neighbors using Jaccard or rank
    similarity.

    :param indices1: The first array of indices of shape (N, D).
    :param indices2: The second array of indices of shape (N, D).
    :return: The average similarity.
    """
    if metric == JACCARD:
        return _jaccard_sim(indices1, indices2)
    else:
        return _rank_sim(indices1, indices2)


def _nearest_neighbors(
    embeds1: np.ndarray,
    embeds2: np.ndarray,
    queries1: np.ndarray,
    queries2: np.ndarray,
    metric: str,
    k: int,
    nn_function: str,
    baseline: bool = False,
):
    """
    Calculates the nearest neighbors for two sets of queries and returns the k indices of the closest embeddings for
    each query. After obtaining the indices, their similarity is calculated.

    :param embeds1: The first set of embeddings of shape (N, D1).
    :param embeds2: The second set of embeddings of shape (N, D2).
    :param queries1: The first set of queries of shape (M, D1).
    :param queries2: The second set of queries of shape (M, D2).
    :param metric: The metric to use for comparing nearest neighbors.
    :param k: The number of nearest neighbors to retrieve.
    :param nn_function: The metric to use for finding nearest neighbors.
    :param baseline: Whether to compute a baseline score, default: False.
    :return: A list of similarity scores of length k.
    """
    neigh1 = NearestNeighbors(n_neighbors=k, metric=nn_function, algorithm="brute")
    neigh1.fit(embeds1)

    neigh2 = NearestNeighbors(n_neighbors=k, metric=nn_function, algorithm="brute")
    neigh2.fit(embeds2)

    indices1 = neigh1.kneighbors(queries1, n_neighbors=k, return_distance=False)
    indices2 = neigh2.kneighbors(queries2, n_neighbors=k, return_distance=False)

    if baseline:
        np.random.shuffle(indices1)

    sims = []

    for i in range(k):
        j = i + 1
        sims.append(nn_sim(indices1[:, :j], indices2[:, :j], metric))
    return sims, None


# Code taken from https://haydn.fgl.dev/posts/a-better-index-of-similarity/
def _cka(A: torch.Tensor, B: torch.Tensor) -> float:
    # Mean center each neuron
    A = A - torch.mean(A, dim=0, keepdim=True)
    B = B - torch.mean(B, dim=0, keepdim=True)

    dot_product_similarity = torch.linalg.norm(torch.matmul(A.t(), B)) ** 2

    normalization_x = torch.linalg.norm(torch.matmul(A.t(), A))
    normalization_y = torch.linalg.norm(torch.matmul(B.t(), B))

    cka = dot_product_similarity / (normalization_x * normalization_y)

    dot_product_similarity.detach()
    normalization_x.detach()
    normalization_y.detach()
    A.detach()
    B.detach()
    del dot_product_similarity, normalization_x, normalization_y, A, B
    return cka, None


def _calculate_embed_metric(
    embeds1: torch.Tensor,
    embeds2: torch.Tensor,
    queries1: np.ndarray,
    queries2: np.ndarray,
    metric: str,
    batch_size: int,
    device: torch.device,
    k: int,
    nn_function: str,
    baseline: bool = False,
):
    """
    Calculates the similarity between two embedding matrices using the given metric.

    :param embeds1: Document embedding tensor of shape (N, D1).
    :param embeds2: Document embedding tensor of shape (N, D1) if the metric requires matching dimensions or (N, D2).
    :param queries1: Embedding of queries of shape (N, D1).
    :param queries2: Embedding of queries of shape (N, D2).
    :param metric: The metric to be used, required to be one of AVAILABLE_METRICS.
    :param batch_size: The batch size used to calculate pairwise similarity.
    :param device: The device on which calculations should be performed.
    :param k: The number of nearest neighbors to retrieve for Jaccard or rank similarity.
    :param nn_function: The function for determining nearest neighbors.
    :param baseline: Whether to compute a baseline score, default: False.
    :return: The calculated similarity.
    """
    assert len(embeds1) == len(embeds2)

    if metric in MATCH_DIM_METRICS:
        return _mean_pairwise_similarity(embeds1, embeds2, metric, batch_size, device)
    elif metric in NEAREST_NEIGHBORS:
        return _nearest_neighbors(
            np.array(embeds1),
            np.array(embeds2),
            queries1,
            queries2,
            metric,
            k,
            nn_function,
            baseline,
        )
    elif metric == CKA:
        return _cka(embeds1, embeds2)
    else:
        raise NotImplementedError(f"Provided unsupported metric {metric} for embedding similarity!")
################################ DONE ################################

In [74]:
import jaxtyping
from pydantic import ConfigDict
import json

class Table(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    #
    table_name: str
    # Dataset <-> IDX
    dataset2idx: Dict[str, int] = {}
    idx2dataset: Dict[int, str] = {}
    # Model <-> IDX
    model2idx: Dict[str, int] = {}
    idx2model: Dict[int, str] = {}
    # Number of layers, i.e. 0, 1, ... (0 = OLS)
    n_layers2idx: Dict[int, int] = {}
    idx2n_layers: Dict[int, int] = {}

    # The actual values
    table: Optional[jaxtyping.Float[torch.Tensor, "dataset model model n_layers"]] = None

    def save_to_folder_path(self, save_path: Path):
        assert not save_path.is_file(), f"Save path {save_path} is a file, not a directory"
        save_path.mkdir(parents=True, exist_ok=False)
        save_file({"table": self.table}, save_path / f"table.safetensors")
        with open(save_path / f"info.json", "w") as f:
            model = self.model_copy()
            model.table = None
            f.write(model.model_dump_json())
    
    @staticmethod
    def load_from_folder_path(load_path: Path):
        assert load_path.is_dir(), f"Load path {load_path} is not a directory"
        with open(load_path / f"info.json", "r") as f:
            info = json.load(f)
        table = load_file(load_path / f"table.safetensors")["table"]
        return Table(**info, table=table)

In [ ]:
DATASET2IDX = {dataset: idx for idx, dataset in enumerate(DATASETS)}
IDX2DATASET = {idx: dataset for dataset, idx in DATASET2IDX.items()}
MODEL2IDX = {model: idx for idx, model in enumerate(MODEL_NAMES)}
IDX2MODEL = {idx: model for model, idx in MODEL2IDX.items()}
N_LAYERS2IDX = {n_layer: idx for idx, n_layer in enumerate(N_LAYERS)}
IDX2N_LAYERS = {idx: n_layer for n_layer, idx in N_LAYERS2IDX.items()}
MODEL2IDX["Mean Estimator"] = len(MODEL_NAMES)
IDX2MODEL[len(MODEL_NAMES)] = "Mean Estimator"
MODEL2IDX["N(0,1) Control"] = len(MODEL_NAMES)
IDX2MODEL[len(MODEL_NAMES)] = "N(0,1) Control"

In [66]:
diagonalizer = einops.rearrange(torch.eye(len(MODEL_NAMES)), "i j -> 1 i j 1") # Helper
assert diagonalizer.shape == (1, len(MODEL_NAMES), len(MODEL_NAMES), 1)
#################################### ALL THE TABLES WE ARE GOING TO WANT TO VISUALIZE ################################
NUM_QUERIES = 25
ckas_table_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), 1) # NOTE natives = 1
ckas_table_natives[:] += 2 * diagonalizer
assert ckas_table_natives[0, 0, 0, 0] == 1
assert ckas_table_natives[0, 1, 0, 0] == -1
assert ckas_table_natives[0, 0, 1, 0] == -1
assert ckas_table_natives[0, 1, 1, 0] == 1
# print(ckas_table_natives[:2, :2, :2, 0])
ckas_uncentered_tables_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), 1) # NOTE natives = 1
ckas_uncentered_tables_natives[:] = diagonalizer

def kernel_cka(embeddings1: torch.Tensor, embeddings2: torch.Tensor) -> float:
    # not centered
    embeddings1_kernel = (embeddings1 @ embeddings1.T)
    embeddings2_kernel = (embeddings2 @ embeddings2.T)
    return ((embeddings1_kernel * embeddings2_kernel).sum() / (embeddings1_kernel.norm() * embeddings2_kernel.norm())).item()

device = "cuda:0"
mean_estimator_cka_row_centered = torch.zeros(len(DATASETS), len(MODEL_NAMES), 1, 1) # APPEND immediately after
mean_estimator_cka_row_uncentered = torch.zeros(len(DATASETS), len(MODEL_NAMES), 1, 1) # APPEND immediately after
n01_estimator_cka_row_centered = torch.zeros(len(DATASETS), len(MODEL_NAMES), 1, 1) # APPEND immediately after
n01_estimator_cka_row_uncentered = torch.zeros(len(DATASETS), len(MODEL_NAMES), 1, 1) # APPEND immediately after
for idx, dataset in tqdm.tqdm(list(enumerate(DATASETS))):
    print("LOADING EMBEDDINGS FOR", dataset)
    model2embeddings = get_all_embeddings(embeddings_paths[0], dataset, device)
    n01_embeddings = torch.randn_like(model2embeddings[MODEL_NAMES[0]]) # Picking one is fine
    print("COMPUTING CKA FOR", dataset)
    cka_centered_table = torch.diag(torch.ones(len(MODEL_NAMES)))
    cka_uncentered_table = torch.diag(torch.ones(len(MODEL_NAMES)))
    for i in range(len(MODEL_NAMES)): # SRC
        for j in range(i + 1, len(MODEL_NAMES)): # TGT
            # Centered
            cka_centered_table[i, j], _ = _cka(model2embeddings[MODEL_NAMES[i]], model2embeddings[MODEL_NAMES[j]])
            cka_centered_table[j, i] = cka_centered_table[i, j]
            # Uncentered
            cka_uncentered_table[i, j] = kernel_cka(model2embeddings[MODEL_NAMES[i]], model2embeddings[MODEL_NAMES[j]])
            cka_uncentered_table[j, i] = cka_uncentered_table[i, j]
    ckas_table_natives[idx, :, :, 0] = cka_centered_table
    ckas_uncentered_tables_natives[idx, :, :, 0] = cka_uncentered_table
    # Find centered and non-centered ckas
    # for 
    # for model_name, embeddings in model2embeddings.items():
    #     for layer_idx in range(len(N_LAYERS)):
    #         embeddings_layer = embeddings[layer_idx]
    #         embeddings_layer_normalized = embeddings_layer / embeddings_layer.norm(dim=-1, keepdim=True)
    #         embeddings_layer_normalized_reshaped = einops.rearrange(embeddings_layer_normalized, "b d -> b 1 d 1")
    #         ckas_table_natives[dataset_idx, model_idx, model_idx, layer_idx] = 1
    pass


# NOTE: here we try to understand what is going on... it seems like 
# model2embeddings = get_all_embeddings(embeddings_paths[0], DATASETS[0], device)
# embeddings1, embeddings2 = model2embeddings[MODEL_NAMES[0]], model2embeddings[MODEL_NAMES[1]]
# embeddings1 -= embeddings1.mean(dim=0, keepdim=True)
# embeddings2 -= embeddings2.mean(dim=0, keepdim=True)
# embeddings1_kernel = (embeddings1 @ embeddings1.T)
# embeddings2_kernel = (embeddings2 @ embeddings2.T)
# # embeddings1_kernel -= embeddings1_kernel.mean(dim=0, keepdim=True)
# # embeddings2_kernel -= embeddings2_kernel.mean(dim=0, keepdim=True)
# correlation = (embeddings1_kernel * embeddings2_kernel).sum() / (embeddings1_kernel.norm() * embeddings2_kernel.norm())
# print("cka for embeddings:", _cka(embeddings1, embeddings2))
# print("cka for kernels:", correlation) # NOTE: not sure why we want to do this centerd?



  0%|          | 0/6 [00:00<?, ?it/s]

LOADING EMBEDDINGS FOR arguana
COMPUTING CKA FOR arguana


 17%|█▋        | 1/6 [00:03<00:18,  3.69s/it]

LOADING EMBEDDINGS FOR fiqa
COMPUTING CKA FOR fiqa


 33%|███▎      | 2/6 [00:26<01:00, 15.22s/it]

...

 67%|██████▋   | 4/6 [00:52<00:24, 12.26s/it]

LOADING EMBEDDINGS FOR hotpotqa
COMPUTING CKA FOR hotpotqa


 83%|████████▎ | 5/6 [01:08<00:13, 13.47s/it]

LOADING EMBEDDINGS FOR trec-covid
COMPUTING CKA FOR trec-covid


100%|██████████| 6/6 [01:30<00:00, 15.16s/it]


In [ ]:
# Plot the CKA tables
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('CKA Similarity Matrices Across Datasets')

for idx, dataset in enumerate(DATASETS):
    # Plot centered CKA
    ax = axes[0, idx]
    sns.heatmap(ckas_table_natives[idx, :, :, 0], 
                ax=ax,
                cmap='viridis',
                vmin=0, 
                vmax=1,
                xticklabels=MODEL_NAMES,
                yticklabels=MODEL_NAMES)
    ax.set_title(f'{dataset} - Centered')
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
    
    # Plot uncentered CKA  
    ax = axes[1, idx]
    sns.heatmap(ckas_uncentered_tables_natives[idx, :, :, 0],
                ax=ax, 
                cmap='viridis',
                vmin=0,
                vmax=1, 
                xticklabels=MODEL_NAMES,
                yticklabels=MODEL_NAMES)
    ax.set_title(f'{dataset} - Uncentered')
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

In [76]:



n01_mse_row = -torch.ones(len(DATASETS), len(MODEL_NAMES), 1, 1)
variances_row = -torch.ones(len(DATASETS), len(MODEL_NAMES), 1, 1)

for dataset in DATASETS:

In [ ]:
ckas_centered_table_obj = Table(
    table_name="ckas_centered_table_natives",
    dataset2idx=DATASET2IDX,
    idx2dataset=IDX2DATASET,
    model2idx=MODEL2IDX, 
    idx2model=IDX2MODEL, 
    n_layers2idx=N_LAYERS2IDX, 
    idx2n_layers=IDX2N_LAYERS, 
    table=ckas_table_natives
)
ckas_uncentered_table_obj = Table(
    table_name="ckas_uncentered_table_natives", 
    dataset2idx=DATASET2IDX,
    idx2dataset=IDX2DATASET,
    model2idx=MODEL2IDX, 
    idx2model=IDX2MODEL, 
    n_layers2idx=N_LAYERS2IDX, 
    idx2n_layers=IDX2N_LAYERS, 
    table=ckas_uncentered_tables_natives
)

save_path_centered = Path("/mnt/align3_drive/adrianoh/dl_final_project_tables/cka_centered_natives")
save_path_uncentered = Path("/mnt/align3_drive/adrianoh/dl_final_project_tables/cka_uncentered_natives")
ckas_centered_table_obj.save_to_folder_path(save_path_centered)
ckas_uncentered_table_obj.save_to_folder_path(save_path_uncentered)

In [ ]:
"""
Get all the damn MSEs!!!!!
"""
mses_table = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
mses_table[:] = diagonalizer
for dataset in DATASETS:
    embeddings = get_all_embeddings(embeddings_paths[0], dataset, device)
    for layer_idx in range(len(N_LAYERS)):
        

In [ ]:
ckas_table_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
ckas_table_stitched[:] = diagonalizer
ckas_uncentered_tables_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
ckas_uncentered_tables_stitched[:] = diagonalizer


In [ ]:
# JACCARD and rank - natives
jaccard_1_tables_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_1_tables_natives[:] = diagonalizer
jaccard_5_tables_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_5_tables_natives[:] = diagonalizer
jaccard_10_tables_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_10_tables_natives[:] = diagonalizer
jaccard_100_tables_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_100_tables_natives[:] = diagonalizer
jaccard_1000_tables_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_1000_tables_natives[:] = diagonalizer
rankings_tables_natives = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
rankings_tables_natives[:] = diagonalizer

In [ ]:
# JACCARD and rank - stitched
jaccard_1_tables_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_1_tables_stitched[:] = diagonalizer
jaccard_5_tables_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_5_tables_stitched[:] = diagonalizer
jaccard_10_tables_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_10_tables_stitched[:] = diagonalizer
jaccard_100_tables_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_100_tables_stitched[:] = diagonalizer
jaccard_1000_tables_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
jaccard_1000_tables_stitched[:] = diagonalizer
rankings_tables_stitched = -torch.ones(len(DATASETS), len(MODEL_NAMES), len(MODEL_NAMES), len(N_LAYERS))
rankings_tables_stitched[:] = diagonalizer